# Data Cleaning Notebook
- utilize insights from EDA notebook to clean data
- construct target field
- drop redundant and highly correlated fields
- handle null/missing values
- log transform fields with heavy tails

In [56]:
import pandas as pd
import numpy as np

In [58]:
df = pd.read_csv("../data/bitcoin.csv", parse_dates=["Date"])
df = df.sort_values("Date").reset_index(drop=True)

In [60]:
# trim early zero price period
df = df[df['btc_market_price'] > 0].copy()

# construct t+7 target
df['target_t7'] = df['btc_market_price'].shift(-7)
df = df.dropna(subset=['target_t7'])

In [62]:
# drop redundant / irrelevant fields
drop_cols = [
    "btc_market_cap",
    "btc_estimated_transaction_volume_usd",
    "btc_cost_per_transaction_percent",
    "btc_n_transactions_total",
    "btc_n_orphaned_blocks",  
]
df = df.drop(columns=drop_cols, errors="ignore")

# collapse correlated groups
df = df.drop(columns=[
    "btc_blocks_size",  # keep avg_block_size
    "btc_difficulty",   # keep hash_rate
    "btc_miners_revenue", # keep transaction_fees
    "btc_n_transactions_excluding_popular",
    "btc_n_transactions_excluding_chains_longer_than_100",
    "btc_n_transactions_per_block"
], errors="ignore")

In [66]:
# impute missing btc_trade_volume values
df["btc_trade_volume"] = df["btc_trade_volume"].ffill()

In [68]:
# save cleaned dataset
df.to_csv("../data/bitcoin_clean.csv", index=False)

df.head(10)

,Date,btc_market_price,btc_total_bitcoins,btc_trade_volume,btc_avg_block_size,btc_median_confirmation_time,btc_hash_rate,btc_transaction_fees,btc_cost_per_transaction,btc_n_unique_addresses,btc_n_transactions,btc_output_volume,btc_estimated_transaction_volume,target_t7
175,2010-08-17,0.076900,3744250.0,923.0018,0.000959,0.0,0.003333,0.67,1.430952,393.0,352.0,72855.15,48276.0,0.066889
176,2010-08-18,0.074000,3750900.0,206.7786,0.001973,0.0,0.003384,1.56,1.305501,449.0,377.0,52829.65,35791.0,0.066500
177,2010-08-19,0.068800,3757900.0,51.8784,0.000715,0.0,0.003562,0.00,1.308696,395.0,368.0,32027.42,13756.0,0.066499
178,2010-08-20,0.066700,3766250.0,293.9825,0.000649,0.0,0.004249,0.00,1.399359,433.0,398.0,36647.52,21143.0,0.065000
179,2010-08-21,0.066899,3775450.0,731.0702,0.000528,0.0,0.004681,0.00,1.768594,396.0,348.0,33790.41,17456.0,0.065000
180,2010-08-22,0.066400,3785400.0,1118.9382,0.000491,0.0,0.005063,0.00,1.887657,358.0,350.0,45810.46,29971.0,0.064800
181,2010-08-23,0.066000,3796250.0,300.8243,0.000495,0.0,0.005521,0.00,1.840874,431.0,389.0,29274.46,11325.0,0.069000
182,2010-08-24,0.066889,3806500.0,469.8232,0.000434,0.0,0.005215,0.00,2.034458,393.0,337.0,28624.03,11639.0,0.064970
183,2010-08-25,0.066500,3817850.0,296.0216,0.000772,0.0,0.005775,0.80,1.905997,414.0,396.0,24234.76,7927.0,0.064900
184,2010-08-26,0.066499,3828250.0,270.9007,0.000499,0.0,0.005292,0.00,1.849170,391.0,374.0,30534.45,18406.0,0.062900


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2738 entries, 175 to 2912
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Date                              2738 non-null   datetime64[ns]
 1   btc_market_price                  2738 non-null   float64       
 2   btc_total_bitcoins                2738 non-null   float64       
 3   btc_trade_volume                  2738 non-null   float64       
 4   btc_avg_block_size                2738 non-null   float64       
 5   btc_median_confirmation_time      2738 non-null   float64       
 6   btc_hash_rate                     2738 non-null   float64       
 7   btc_transaction_fees              2738 non-null   float64       
 8   btc_cost_per_transaction          2738 non-null   float64       
 9   btc_n_unique_addresses            2738 non-null   float64       
 10  btc_n_transactions                2738 non-null   f

In [72]:
df.describe(include="all").transpose()

,count,mean,min,25%,50%,75%,max,std
Date,2738,2014-05-16 12:00:00,2010-08-17 00:00:00,2012-07-01 06:00:00,2014-05-16 12:00:00,2016-03-30 18:00:00,2018-02-13 00:00:00,NaN
btc_market_price,2738.0,930.377008,0.0609,11.222022,252.71,617.921563,19498.683333,2419.879152
btc_total_bitcoins,2738.0,12059588.381118,3744250.0,9355512.5,12779187.5,15377793.75,16863312.5,3696172.889257
btc_trade_volume,2738.0,84430257.722668,41.0628,440835.509475,11739370.21,30705626.232875,5352015515.54,315777772.920033
btc_avg_block_size,2738.0,0.376779,0.000434,0.076631,0.229098,0.715818,1.110327,0.354778
btc_median_confirmation_time,2738.0,8.028042,0.0,6.683333,8.091667,10.579167,47.733333,4.724338
btc_hash_rate,2738.0,1423210.779134,0.003333,13.916822,71727.481822,1250355.732292,24293141.3193,3306748.836303
btc_transaction_fees,2738.0,64.805983,0.0,11.606229,23.806326,53.855739,1495.946477,120.231304
btc_cost_per_transaction,2738.0,15.963187,0.13486,5.057195,8.207428,16.595948,161.686071,21.430689
btc_n_unique_addresses,2738.0,207860.886413,284.0,25743.5,146181.5,375081.5,1072861.0,209820.362728
